In [41]:
import numpy as np
import pandas as pd
from pandas.core.indexes.base import maybe_extract_name
import river

from river import compose
from river import preprocessing
from river import linear_model
from river import metrics
from river import evaluate
from river import neighbors
from sklearn.ensemble import RandomForestClassifier

from kafka import KafkaConsumer
import skmultiflow


In [42]:
# write the print content to file 
f = open("./2-1-6 predictive scores for an adaptive model with initial data.log",'w+')
f2 = open("./2-1-6 predictive scores for an adaptive mode with kafka data.log",'w+')


In [43]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ get data from sklearn ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# read from sklearn


data_aftcat = pd.read_csv(r"./2-1-1 5388 data_aftcat.csv")
data_y = pd.read_csv(r"./2-1-1 5388 data_aftcat label.csv")
data = pd.read_csv(r"./2-1-1 5388 data_aftcat_full_no label.csv")

d_col = data.columns
y_col = data_y.columns
# change from pd.DataFrame to np.array
data = np.array(data)
data_y = np.array(data_y)

In [44]:
# ^^^^^^^^^^^^^^^^^^^^^ train a model using river ^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# train an adaptive model and prediction
mm = river.preprocessing.MinMaxScaler()
#selector = river.feature_selection.VarianceThreshold()
#model_adaptive = tree.HoeffdingAdaptiveTreeClassifier(grace_period=100,split_confidence=1e-5,leaf_prediction='nb',nb_threshold=10,seed=0)
model_adaptive = neighbors.KNNClassifier() # window_size (int) – defaults to 1000
#model = compose.Pipeline(mm,selector,model_adaptive)
model = compose.Pipeline(mm,model_adaptive)
#model = river.imblearn.RandomOverSampler((mm | model_adaptive),desired_dist={False: 0.5, True: 0.5})


In [45]:
metric_acc = metrics.Accuracy()
metric_f1 = metrics.F1()


In [46]:
i = 0
for xi, yi in zip(data, data_y):
    xi = dict(zip(d_col, xi))   # transfor array to dict
    #yi = dict(zip(y_col,yi))
    yi = yi[0]
    #print(yi)
    #xi_scaled = model.learn_one(xi).transform_one(xi)
    yi_pred = model.predict_one(xi)  # make a prediction
    metric_acc = metric_acc.update(yi,yi_pred) # update the metric
    metric_f1 = metric_f1.update(yi,yi_pred)
    model = model.learn_one(xi,yi) # make the model learn
    i=i+1
print("metric_acc:",metric_acc)
print("metric_f1",metric_f1)
print("i:",i)

print("metric_acc:",metric_acc,file=f)
print("metric_f1",metric_f1,file=f)
print("i:",i,file=f)


metric_acc: Accuracy: 96.97%
metric_f1 F1: 0.833037
i: 24745


In [47]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ read kafka data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# instantiate the KafkaConsumer Class using the arguments mentioned.
# do not change any arguments other than the first positional argument.
consumer = KafkaConsumer(
    'task1',   # change this to "task2" for the IOT Botnet Detection  ---- important ----
    bootstrap_servers="34.130.121.39:9092",
    sasl_plain_username="student",
    sasl_plain_password="uottawa",
    security_protocol="SASL_PLAINTEXT",
    sasl_mechanism="PLAIN",
    auto_offset_reset='earliest',
    enable_auto_commit=False
)

In [48]:
metric_acc_kafka = metrics.Accuracy()
metric_f1_kafka = metrics.F1()

In [49]:
# Data Stream flowing in. - - - Dict()
i = 0
# concept drift detection
kswin = river.drift.KSWIN(window_size = 1000)
drifts_ksw = []
nan=0
inf=0
for message in consumer:
    #print(message)
    tmp_data=eval(message.value.decode(errors='ignore'))   #dict()
    if (tmp_data['Label']=='BENIGN'):
        dict_y = 0
    else:
        dict_y = 1        
    #print("dict_y:",dict_y)
    del tmp_data['Label']
    dict_data = tmp_data
    #print("dict_data",dict_data)
    #print("items:",tmp_data.items())
    #print("values:",tmp_data.values())

    # prediction and train an adaptive model
    yi_pred = model.predict_one(dict_data)  # make a prediction
    metric_acc_kafka = metric_acc_kafka.update(dict_y,yi_pred) # update the metric
    metric_f1_kafka = metric_f1_kafka.update(dict_y,yi_pred)
    model = model.learn_one(dict_data,dict_y) # make the model learn

    # concept drift detection
    in_drift, in_warning = kswin.update(message.value)
    if in_drift:
        print(f'Drift detected at index {i}')
        print(f'Drift detected at index {i}',file=f2)
        drifts_ksw.append(i)
        kswin.reset()   # As a best practice, we reset the detector   

    if i == 100000:
       break       
    i += 1

Drift detected at index 1181
Drift detected at index 4310
Drift detected at index 5414
Drift detected at index 7014
Drift detected at index 8650
Drift detected at index 9690
Drift detected at index 10985
Drift detected at index 12855
Drift detected at index 13966
Drift detected at index 15550
Drift detected at index 16785
Drift detected at index 17816
Drift detected at index 19297
Drift detected at index 20466
Drift detected at index 22718
Drift detected at index 24039
Drift detected at index 26119
Drift detected at index 27244
Drift detected at index 28646
Drift detected at index 29712
Drift detected at index 30886
Drift detected at index 31986
Drift detected at index 33104
Drift detected at index 35332
Drift detected at index 36373
Drift detected at index 37497
Drift detected at index 38844
Drift detected at index 39952
Drift detected at index 41529
Drift detected at index 42608
Drift detected at index 44136
Drift detected at index 45166
Drift detected at index 46963
Drift detected a

In [50]:
print("metric_acc:",metric_acc_kafka)
print("metric_f1",metric_f1_kafka)
print("i:",i)

print("metric_acc:",metric_acc_kafka,file=f2)
print("metric_f1",metric_f1_kafka,file=f2)
print("i:",i,file=f2)

metric_acc: Accuracy: 96.80%
metric_f1 F1: 0.837577
i: 100000
